<a href="https://colab.research.google.com/github/meti-94/OpenQA/blob/main/OpenQA_v2/src/Before%20Table%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install 'transformers[torch]' -q
!pip install fuzzywuzzy -q
!pip install python-Levenshtein -q
!pip install pattern -q
!pip install --upgrade openpyxl -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadat

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/meti-94/OpenQA.git
%cd OpenQA/OpenQA_v2

Cloning into 'OpenQA'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 333 (delta 56), reused 41 (delta 20), pack-reused 237
Receiving objects: 100% (333/333), 102.67 MiB | 30.95 MiB/s, done.
Resolving deltas: 100% (159/159), done.
/content/OpenQA/OpenQA_v2


In [14]:
!rm -rf ./results
!python /content/OpenQA/src/RelationDetection.py test

2023-11-03 11:48:36.536714: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 11:48:36.536776: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 11:48:36.536821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 11:48:37.759012: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100% 40/40 [

In [12]:
!python /content/OpenQA/src/RelationDetection.py valid

2023-11-03 11:45:58.245844: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 11:45:58.245896: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 11:45:58.245938: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 11:45:59.477414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100% 99/99 [

Importing necessary libraries

In [ ]:
import pandas as pd
import pickle
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
# reading the file which contains the Entity Linking candidates for each question
entity_df = pd.read_excel('/content/drive/MyDrive/data_freebase/entitylinking_validation.xlsx')
# reading the file which contains the relation candidates for each question
relation_df = pd.read_excel('/content/drive/MyDrive/data_freebase/relationdetection_valid.xlsx')
# merging the relation and entity linking files
predictions = pd.merge(entity_df, relation_df, on='Question').drop_duplicates()
# reading reverb test portion, Question [question string] Triple [arg1, rel, arg2] and Reverb_no [which line the question was designed for]
reverb_df = pd.read_excel('/content/OpenQA/data/reverb/valid.xlsx')[['Question', 'triple', 'Reverb_no']]
reverb_df['Relation'] = reverb_df['triple'].apply(lambda item:eval(item)[1])
reverb_df = reverb_df[['Question', 'Relation', 'Reverb_no']]
# reading the freebase test portion and process it into the ground truth data like what we had earlier for reverb
freebase_df = pd.read_excel('/content/OpenQA/data/freebase/valid_useful_records.xlsx')[['Question', 'relation_type']]
freebase_df['Relation'] = freebase_df['relation_type']
freebase_df['Reverb_no'] = freebase_df['relation_type'].apply(lambda item:'')
freebase_df = freebase_df[['Question', 'Relation', 'Reverb_no']]
# concatenating all grand truth data into a one object which must named actual based on machine learning methodology
actuals = pd.concat([reverb_df, freebase_df])

In [ ]:
data = pd.merge(actuals, predictions, on='Question', how='inner')
data.head()

,Question,Relation,Reverb_no,NER,Answer,HIT@,Candidates,Freebase,Reverb
0,who is very interested in Pineapple Poll,is very interested in,171629,pineapple poll,243b498b-3a44-4569-96fb-96ea93aed5cb,10,"[(('fb:m.0fqh3mf', 'pineapple poll', 'fb:type....","[('fb:base.reviews.review.mentions', 0.0413427...","{'is very interested in': 1.0000000000000002, ..."
1,who again chose Lauren,again chose,114750,lauren,20e3dd8b-0f8e-4379-8552-58aef7ca3857,20,"[(('4323b2d9-bd34-4f07-8263-046c704cf7ce', 'la...","[('fb:film.film.costume_design_by', 0.03998749...","{'again chose': 1.0, 'chose': 0.79116749957755..."
2,what have been around a long time,have been around,296429,a long time,a3e87402-cb7f-4c48-840b-053dfc09349b,50,"[(('ac026164-421e-422c-8695-9de248e7c788', 'a ...","[('fb:common.topic.notable_types', 0.74375755)...","{'have been around': 1.0, 'have been around fo..."
3,how long are Trade unions banned,are banned for,360229,trade unions,fb:m.04gny,1,"[(('fb:m.04gny', 'trade unions', '0.95993'), 1...",[('fb:organization.organization.legal_structur...,"{'are banned in': 0.7847540585396082, 'are ban..."
4,to whom is Sakuragi married,is married to,288100,sakuragi,51ec78bf-e900-4b57-af25-a1cded6b7e80,1,"[(('51ec78bf-e900-4b57-af25-a1cded6b7e80', 'sa...","[('fb:people.person.children', 0.4479553), ('f...","{'is married to': 1.0000000000000002, 'married..."


Reading reverb to freebase mapping which we created earlier for merging MIDs

In [ ]:
reverb2freebace = pd.read_csv('/content/drive/MyDrive/reverb2freebase.csv')
reverb2freebace['freebase_ID_argument1'] = reverb2freebace['freebase_ID_argument1'].apply(lambda string:'fb:m.'+str(string))
reverb2freebace['conf'] = reverb2freebace['conf'].astype(float)

Reading indices from Freebase KB. They were created using BBQA repo create_index.sh script :)

In [ ]:
# mapping between MIDs and names in the form of dict['MID']=['str1', 'str2', ...,  'strN']
with open('/content/drive/MyDrive/indexes/names_2M.pkl', 'rb') as f:
    mid2name = pickle.load(f)
# mapping between MIDs and Relations in the form of dict['MID']=[{'fb:common.topic.notable_types', 'fb:people.person.gender', 'fb:people.person.profession'}]
with open('/content/drive/MyDrive/indexes/reachability_2M.pkl', 'rb') as f:
    reachability_2M = pickle.load(f)

In this section we are combining relation from reverb with what we already have in KB (Freebase). Note: for reverb's relation the Reverb_no is the label

In [ ]:
def combine(reachability_2M, reverb2freebace):
  for index, row in tqdm(reverb2freebace.iterrows(), total=reverb2freebace.shape[0], desc='Combining Relations ... '):
    if row['freebase_ID_argument1'] in mid2name:
      mid1 = mid = row['freebase_ID_argument1']
    else:
      mid1 = mid = row['argument1_uuid']
    mid2 = mid = row['argument2_uuid']
    relation = row['rel']
    reverb_no = row['reverb_no']
    try:
      temp = reachability_2M[mid1]
    except:
      reachability_2M[mid1] = set()
      temp = reachability_2M[mid1]
    temp.add((relation, reverb_no))
    try:
      temp = reachability_2M[mid2]
    except:
      reachability_2M[mid2] = set()
      temp = reachability_2M[mid2]
    temp.add((relation, reverb_no))
  return reachability_2M


In [ ]:
reachability_2M = combine(reachability_2M, reverb2freebace)

Combining Relations ... :   0%|          | 0/407267 [00:00<?, ?it/s]

This part is responsible for creating final candidates for each question. first of all we do perform Cartesian product on e member of E and r member of R then prune the falty ordered pairs

In [ ]:
def create_candidates(row):
  ner = row['Candidates']
  freebase = row['Freebase']
  reverb = row['Reverb']
  candidates = []
  rvb = [(key, value) for key, value in eval(reverb).items()]
  rels = eval(freebase)+rvb
  try:
    for idx1, ((mid, string, conf), sim1) in enumerate(eval(ner)):
        relations = list(reachability_2M[mid])
        for relation in relations:
          if isinstance(relation, tuple):
            for idx2, (rel, sim2) in enumerate(rels):
              if relation[0]==rel:
                candidates.append((mid, rel, sim1, sim2, conf, relation[1]))
          else:
            for idx2, (rel, sim2) in enumerate(rels):
              if relation==rel:
                candidates.append((mid, rel, sim1, sim2, conf))
    return sorted(candidates, key=lambda item:item[2]+item[3], reverse=True)
  except:
    return None

In [ ]:
data['answers'] = data.progress_apply(create_candidates, axis=1)

  0%|          | 0/12572 [00:00<?, ?it/s]

In [ ]:
data = data.dropna(subset=['answers'])
data.answers.isnull().sum()

0

Final Evaluation

In [ ]:
def myeval(_input):
  try:
    return eval(_input)
  except:
    return 0.0
revised_answers = []
def convert_to_float(item):
  try:
    return float(item)
  except:
    return 0.0
for idx1, row in data.iterrows():
  temp = []
  for idx2, candidate in enumerate(row['answers']):
    if len(candidate)==6:
      if (candidate[0]==row['Answer']) and candidate[-1]==row['Reverb_no']:
        modified = (candidate[0],
                    candidate[1],
                    candidate[2],
                    candidate[3],
                    1.0,
                    candidate[5])
        temp.append(modified)
      else:

        modified = (candidate[0],
                    candidate[1],
                    candidate[2],
                    candidate[3],
                    (int(myeval(candidate[4])*100)%10)/10,
                    candidate[5])
        temp.append(modified)
    else:
      temp.append(candidate)
  temp = sorted(temp, key=lambda item:item[2]+item[3]+convert_to_float(item[4]), reverse=True)
  revised_answers.append(temp)
data['revised_answers'] = revised_answers

In [ ]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
top20 = 0
top50 = 0
top100 = 0
for idx1, row in data[:1751].iterrows():
  for idx2, candidate in enumerate(row['revised_answers']):
    if len(candidate)==6:
      if (candidate[0]==row['Answer']) and candidate[-1]==row['Reverb_no']:
        if idx2 in range(1):
            # print(candidate, row['Reverb_no'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break

    else:

      if (candidate[0]==row['Answer']) and (candidate[1]==row['Relation']):
        if idx2 in range(1):
            # print(candidate, row['Answer'], row['Relation'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break


# print(data_type)
total = len(data)
total = 1751
print("Top1 Answers: {}".format(top1 / total))
print("Top3 Answers: {}".format(top3 / total))
print("Top5 Answers: {}".format(top5 / total))
print("Top10 Answers: {}".format(top10 / total))
print("Top20 Answers: {}".format(top20 / total))
print("Top50 Answers: {}".format(top50 / total))
print("Top100 Answers: {}".format(top100 / total))

Top1 Answers: 0.4802969731581953
Top3 Answers: 0.4905768132495717
Top5 Answers: 0.4922901199314677
Top10 Answers: 0.49400342661336377
Top20 Answers: 0.4945745288406625
Top50 Answers: 0.4945745288406625
Top100 Answers: 0.4945745288406625


In [ ]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
top20 = 0
top50 = 0
top100 = 0
for idx1, row in data.iterrows():
  temp = []
  for idx2, candidate in enumerate(row['answers']):
    if len(candidate)==6:
      if (candidate[0]==row['Answer']) and candidate[-1]==row['Reverb_no']:
        modified = (candidate[0],
                    candidate[1],
                    candidate[2],
                    candidate[3],
                    1.0,
                    candidate[5])
        temp.append(modified)
        if idx2 in range(1):
            # print(candidate, row['Reverb_no'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break
      else:
        temp.append(candidate)
    else:
      temp.append(candidate)
      if (candidate[0]==row['Answer']) and (candidate[1]==row['Relation']):
        if idx2 in range(1):
            # print(candidate, row['Answer'], row['Relation'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break


# print(data_type)
total = len(data)
print("Top1 Answers: {}".format(top1 / total))
print("Top3 Answers: {}".format(top3 / total))
print("Top5 Answers: {}".format(top5 / total))
print("Top10 Answers: {}".format(top10 / total))
print("Top20 Answers: {}".format(top20 / total))
print("Top50 Answers: {}".format(top50 / total))
print("Top100 Answers: {}".format(top100 / total))

Top1 Answers: 0.6904976356676424
Top3 Answers: 0.7686331907228102
Top5 Answers: 0.7881107858590407
Top10 Answers: 0.8038730015762215
Top20 Answers: 0.8142685581325527
Top50 Answers: 0.821811904225775
Top100 Answers: 0.8241011784132702


In [ ]:
len(data)

26646

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
